In [16]:
import numpy as np
from sentence_transformers import SentenceTransformer
from text_untils import *
import pickle
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
with open('output/bm25/texts.pkl', 'rb') as f:
    texts = pickle.load(f)

with open('output/bm25/bm25plus.pkl', 'rb') as f:
    bm25plus = pickle.load(f)

In [18]:
def load_docs_from_file(file_path):
    docs = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                doc = json.loads(line)
                docs.append(doc)
            except json.JSONDecodeError:
                continue

    return docs

docs = load_docs_from_file('output/bm25/docs.jsonl')

In [19]:
query = 'Tổ chức nào thực hiện trách nhiệm thông báo phòng chống khi xảy ra thiên tai'
# original_question = 'Cơ quan có trách nhiệm truyền tin phòng, chống thiên tai'
query = clean_text(query)
query = word_segment(query)
query = remove_stopword(normalize_text(query))

query_bm25 = query.split()
print(query_bm25)

['tổ_chức', 'thực_hiện', 'trách_nhiệm', 'thông_báo', 'phòng_chống', 'xảy', 'thiên_tai']


In [20]:
top_docs = bm25plus.get_top_n(query_bm25, docs, n=100)
bm25_scores = bm25plus.get_scores(query)
doc_scores = np.sort(bm25_scores)[::-1][:100]

In [21]:
for doc_dict, score in zip(top_docs, doc_scores):
    doc_id = doc_dict['id']
    doc_title = doc_dict['title']
    print(f"ID: {doc_id}, Document: {doc_title}, Score: {score}")

ID: 653a89021e06875841d152d9, Document: quy_định trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai, Score: 403.7688526747695
ID: 653a89191e06875841d1533b, Document: trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai huyện quy_định thế_nào, Score: 395.2592986408791
ID: 653a893d1e06875841d15435, Document: quy_định trách_nhiệm tổ_chức thực_hiện kế_hoạch phòng_chống thiên_tai huyện quy_định, Score: 393.19187226801535
ID: 653a8b391e06875841d15be5, Document: quy_định trách_nhiệm thông_báo cháy rừng xảy, Score: 393.19187226801535
ID: 653a9ebe1e06875841d1c26d, Document: cơ_quan trách_nhiệm truyền phòng_chống thiên_tai, Score: 393.19187226801535
ID: 653a86321e06875841d13683, Document: ban chỉ_đạo quốc_gia phòng_chống thiên_tai trách_nhiệm thực_hiện ứng_phó sự_cố thiên_tai tìm_kiếm cứu nạn, Score: 392.5436243939252
ID: 653a9ef51e06875841d1c4ad, Document: phòng_chống thiên_tai, Score: 390.0497013666529
ID: 653a892e1e06875841d153e9, Document: trách_nhiệm tài_chính đối_với quỹ 

In [22]:
query_sbert = query.replace('_', ' ')
print(query_sbert)

tổ chức thực hiện trách nhiệm thông báo phòng chống xảy thiên tai


In [23]:
df = pd.read_json('data/cleaned/temp.jsonl', lines=True)

In [24]:
doc_ids = [doc_dict['id'] for doc_dict in top_docs]
selected_rows = df[df['_id'].isin(doc_ids)]

In [25]:
sentences_title = []
ids_title = []

for idx in selected_rows.index:
    sentences_title.append(selected_rows.at[idx, 'title'])
    ids_title.append(selected_rows.at[idx, '_id'])

In [26]:
embeddings_title = np.load('output/sbert/embeddings.npy')

In [27]:
model_title = SentenceTransformer('keepitreal/vietnamese-sbert')

In [28]:
query_embedding_title = model_title.encode([query_sbert], convert_to_tensor=True)
query_embedding_title = query_embedding_title.cpu()

In [29]:
similarities_title = cosine_similarity(query_embedding_title, embeddings_title)

top_50_indices_title = np.argsort(similarities_title[0])[::-1][:50]

for idx in top_50_indices_title:
    similarity_score = similarities_title[0][idx]
    similar_sentence = sentences_title[idx]
    doc_id = ids_title[idx]
    print(f"ID: {doc_id}, Similarity Score: {similarity_score:.4f}, Sentence: {similar_sentence}")

IndexError: list index out of range